In [1]:
import torch

In [2]:
from model import *

2024-04-27 20:48:21 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from torch.nn.parallel import DataParallel

In [5]:
# Initialize your model
model = Model(device=device)


/home/hparashar/mtech_projects/speech_understanding/SU-PA3/.venv/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [6]:
model = DataParallel(model)

In [7]:
# Define the path to the .pt file
file_path = "models/Best_LA_model_for_DF.pth"
dictModel = torch.load(file_path,map_location=device)
model.load_state_dict(dictModel)

<All keys matched successfully>

In [8]:
model.train()

DataParallel(
  (module): Model(
    (ssl_model): SSLModel(
      (model): Wav2Vec2Model(
        (feature_extractor): ConvFeatureExtractionModel(
          (conv_layers): ModuleList(
            (0): Sequential(
              (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
              (1): Dropout(p=0.0, inplace=False)
              (2): Sequential(
                (0): TransposeLast()
                (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
                (2): TransposeLast()
              )
              (3): GELU(approximate='none')
            )
            (1-4): 4 x Sequential(
              (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
              (1): Dropout(p=0.0, inplace=False)
              (2): Sequential(
                (0): TransposeLast()
                (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
                (2): TransposeLast()
              )
              (3): GELU(approximate='none')
            )
     

In [9]:
#model.eval()

In [10]:
from utills import *

In [11]:
data_dir = 'datasets/Dataset_Speech_Assignment'
customdataset = CustomDataset(data_dir)
customdataloader = DataLoader(customdataset, batch_size=5, shuffle=True)

In [12]:
count = 0

for batch in customdataloader:
    inputs, labels = batch
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    print(outputs)
    print(labels)
    
    if count == 5:
        break
    count += 1

tensor([[ 1.2629, -2.3834],
        [-3.9719,  2.5440],
        [ 5.8046, -2.6153],
        [ 4.7309, -3.9660],
        [ 4.2338, -3.4952]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0, 0, 0, 1, 0], device='cuda:0')
tensor([[ 6.0407, -4.5699],
        [ 6.2589, -4.7950],
        [-4.8919,  3.1835],
        [ 6.0278, -3.4362],
        [ 2.6168, -2.0095]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0, 0, 0, 1, 0], device='cuda:0')
tensor([[ 3.2799, -2.8399],
        [ 2.6723, -2.7086],
        [ 4.0239, -3.6964],
        [-4.9233,  3.0070],
        [ 5.3630, -4.0191]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0, 0, 1, 0, 0], device='cuda:0')
tensor([[ 5.0360, -4.3565],
        [-3.1554,  5.2207],
        [ 7.6295, -5.1979],
        [ 4.5282, -3.5170],
        [ 3.4851, -3.3489]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([1, 0, 1, 1, 1], device='cuda:0')
tensor([[ 3.7131, -2.3872],
        [ 1.4797, -0.8675],
        [ 3.6337, -5.3560],
        [ 5.8365

In [13]:
from tqdm import tqdm

In [14]:
def train(train_loader, model, optimizer, device):
    
    criterion = nn.CrossEntropyLoss()  # No need for custom weights if the classes are balanced
    
    model.train()
    epoch = 5
    running_loss = 0
    num_total = 0.0
    correct_predictions = 0
    
    for i in range(epoch):
        running_loss = 0
        num_total = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {i+1}")
        for batch_x, batch_y in pbar:
            batch_size = batch_x.size(0)
            num_total += batch_size
            
            batch_x = batch_x.to(device)
            batch_y = batch_y.view(-1).type(torch.int64).to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            batch_out = model(batch_x)
            
            # Compute loss
            batch_loss = criterion(batch_out, batch_y)
            
            # Backward pass and optimization
            batch_loss.backward()
            optimizer.step()
            
            running_loss += (batch_loss.item() * batch_size)
            
            # Calculate accuracy
            _, predicted_classes = torch.max(batch_out, 1)
            correct_predictions += (predicted_classes == batch_y).sum().item()
            pbar.set_postfix({'loss': running_loss / num_total, 'accuracy': correct_predictions / num_total})
        running_loss /= num_total
        accuracy = correct_predictions / num_total
        #print(f"Epoch {i+1} - Loss: {running_loss:.4f}, Accuracy: {accuracy:.4f}")


In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train(customdataloader, model, optimizer, device)

Epoch 1:  77%|███████▋  | 46/60 [04:22<01:20,  5.72s/it, loss=0.78, accuracy=0.752] 


KeyboardInterrupt: 